# Laboratoria 9: BERT i atencja


### Zadanie 1 (3 pkt), atencja dekodera względem (en)kodera

Poniżej znajdują się dwie macierze, `encoder_states` oraz `decoder_states` reprezentujące stan warstwy ukrytej po przetworzeniu każdego slowa z enkodera i dekodera. Pojedynczy stan warstwy ukrytej zawiera embedding o dlugosci = 3. W enkoderze mamy 4 stany warstwy ukrytej RNNów, gdyż przetwarzamy sekwencję 4 tokenów.

W dekoderze mamy 5 tokenów, które powinny być wygenerowane z sekwencji przetwarzanej (en)koderem.

Zadanie polega na:
a) Obliczniu podobieństwa wszystkich embeddingów z dekodera (queries) względem wszystkich embeddingów kolejnych stanów (en)kodera (keys) [pamiętajcie, że macierze potrafią w transponowanie. W `NumPy` macierz transponujemy za pomocą `macierz.T`]

b) Na utworzonej macierzy podobieństwa należy wykonać softmax (zaimportowany z scipy). Uwaga:  pamiętajcie, żeby aplikować softmax w dobrym wymiarze. Wszystkie stany ukryte enkodera powinny zostac zasoftmaksowane względem zadanego stanu dekodera, nie odwrotnie. W scipy, funkcja softmax zawiera argument axis, który może pomóc.

c) Należy wykorzystać macierz atencji z kroku b) i `encoder_states` do wygenerowania macierzy zawierającej wektory kontekstu dla każdego tokenu z dekodera.


In [31]:
from numpy import *
from scipy.special import softmax

encoder_states = array(
  # Embedding z warstwy ukrytej enkodera w kroku 1, np. dla slowa Ala
  [[1.2, 3.4, 5.6],
   # Embedding z warstwy ukrytej enkodera w kroku 2, np. dla slowa ma
   [-2.3, 0.2, 7.2],
   # Embedding z warstwy ukrytej enkodera w kroku 3, np. dla slowa kota
   [10.2, 0.2, 0.3],
   # Embedding z warstwy ukrytej enkodera w kroku 4, np. dla token'u <EOS> (koniec sekwencji)
   [0.4, 0.7, 1.2]]
)

decoder_states = array(
  # Embedding z warstwy ukrytej dekodera w kroku 1, np. przed wygenerowaniem slowa Alice
  [[0.74, 0.23, 0.56],
   # Embedding z warstwy ukrytej dekodera w kroku 2, np. przed wygenerowaniem slowa owns
   [7.23, 0.12, 0.55],
   # Embedding z warstwy ukrytej dekodera w kroku 3, np. przed wygenerowaniem slowa a
   [9.12, 4.23, 0.44],
   # Embedding z warstwy ukrytej dekodera w kroku 4, np. przed wygenerowaniem slowa cat
   [4.1, 3.23, 0.5],
   # Embedding z warstwy ukrytej dekodera w kroku 5, np. przed wygenerowaniem slowa cat
   [5.2, 3.1, 8.5]]
)
print(
  probabilities := (encoder_states @ decoder_states.T).T,
  softmax_ := softmax(probabilities, axis=1),
  context := softmax_ @ encoder_states,
  sep='\n'
)


[[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]]
[[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]]
[[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]


**Oczekiwane wartości:**

a) 
[[  4.806   2.376   7.762   1.129]
 [ 12.164 -12.645  73.935   3.636]
 [ 27.79  -16.962  94.002   7.137]
 [ 18.702  -5.184  42.616   4.501]
 [ 64.38   49.86   56.21   14.45 ]] 


b) 
[[4.91780633e-02 4.32948093e-03 9.45248312e-01 1.24414389e-03]
 [1.49003187e-27 2.50486173e-38 1.00000000e+00 2.94803216e-31]
 [1.75587568e-29 6.44090821e-49 1.00000000e+00 1.88369172e-38]
 [4.11416552e-11 1.74069934e-21 1.00000000e+00 2.79811669e-17]
 [9.99716568e-01 4.94220792e-07 2.82937800e-04 2.06801368e-22]] 

c) 
[[ 9.69108631  0.35799187  0.59163688]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [10.2         0.2         0.3       ]
 [ 1.20254471  3.39909302  5.59850122]]
 
 (albo to samo transponowane)


## Zadanie 2 (2 punkty): tokenizacja tekstu 

Korzystając z biblioteki transformers (https://huggingface.co/transformers/) wczytaj tokenizator BERTa (BERT to już wytrenowany (pretrenowany) model, oparty o ideę transformera (a w zasadzie o jego enkoder)). Ponieważ model jest gotowy i można go wykorzystać do generowania embeddingów tokenów, ważnym jest, aby tokenizacja była przeprowadzona identycznie do tego jak podczas trenowania BERTa.

Wybierzmy pretrenowany tokenizator o nazwie `bert-base-uncased` i zobaczmy jaki będzie efekt tokenizacji na tekście zawartym w zmiennej `text_to_tokenize`.

Zwróć uwagę na to, że niektóre rzadkie słowa zostały podzielone na subtokeny -- zgodnie z algorytmem WordPiece jaki omawialiśmy na przedostatnim spotkaniu.


In [36]:
from transformers import BertTokenizer
text_to_tokenize = "I've bought a new GPU last year it was GeForce RTX 3060"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokens = tokenizer.tokenize(text_to_tokenize)

print(tokens)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

['i', "'", 've', 'bought', 'a', 'new', 'gp', '##u', 'last', 'year', 'it', 'was', 'ge', '##force', 'rt', '##x', '306', '##0']


## Zadanie 3 (brak punktów):
Poniżej znajduje się kod wykorzystujący przygotowane wcześniej zmienne `tokenizer` i `tokens` i które dla każdego tokenu z tokens generuje embedding. W odróżnieniu od GloVe, te embeddingi są świadome kontekstu w jakim właśnie występują. 

In [ ]:
from transformers import BertModel
import torch

model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
# nie chcemy trenować modelu, tylko go wykorzystać
model.eval()

# BERT wymaga specjalnych tokenów [CLS] na początku i [SEP] separującego pary zdań (BERT jest trenowany parami zdań)
tokens_with_specials = ['CLS'] + tokens + ['SEP']
# zamiana listy tokenów na listę identyfikatorów (liczb) ze słownika
tokens_with_specials = tokenizer.convert_tokens_to_ids(
  tokens_with_specials)
# zamiana na tensor, opakowanie w batch
tokens_tensor = torch.tensor([tokens_with_specials])

# Wygeneruj maskę mówiącą o tym, które tokeny nalezą do zdania 1, a które do 2. W naszym zadaniu wszystkie tokeny nalezą do zdania 1
segments = torch.tensor([[1] * len(tokens_with_specials)])

with torch.no_grad():
  # wygenerujmy embedding'i BERTem
  outputs = model(tokens_tensor, segments)
  # wez pierwszy batch danych i ostatnią warstwę
  tokens_embeddings = outputs['last_hidden_state'][0]
  # 20x768, mamy 20 (sub)tokenów, (18 właściwych + cls + sep) i każdy mapowany jest na wektor liczb o długości 768
  print(tokens_embeddings.shape)
  # wez embedding pierwszego sub-token'u z sekwencji (przeskakujemy CLS token)
  print(tokens_embeddings[1])